In [15]:
import gensim, time, os, json, pandas, pyltp
from pyltp import SentenceSplitter,NamedEntityRecognizer,Postagger,Parser,Segmentor
import numpy as np

In [16]:
df = pandas.read_csv('AutoMaster_TrainSet.csv')

In [17]:
df

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手
...,...,...,...,...,...,...
82938,Q82939,道奇,酷威,这车正时链条的小链条怎么对，,技师说：稍等上图|车主说：感谢|车主说：大哥？|技师说：[图片]|技师说：[图片]|技师说：...,查阅资料
82939,Q82940,大众,迈腾,2013款迈腾二代ea888怎么简单的判断烧机油？今天发现油气分离器管子拔下来，管头里面全是...,技师说：你好，判断是不是烧机油很简单，一个看看排气管有没有冒蓝烟，特别是早上启动时候，其次就...,根据你的描述， 1.如果车子烧机油首先就是看排气管有没有出现冒蓝烟，其次就是检查机油有没有少...
82940,Q82941,现代,领动,新车800公里 自动1.6 挡踩油门有滋滋声。松油门就没有了。空挡加油也没有滋滋声,技师说：这种情况需要检查一下进气道的翻板电磁阀有没有问题，发电机皮带轮那一块有没有异响，变速...,这种情况需要检查一下底盘护板有无异响，进气道翻板电磁阀有无异响，发动机及皮带轮这一块有没有异...
82941,Q82942,金杯,西部牛仔,微型货车，金杯西部牛仔，4f18发动机。德尔福电脑。 问题 冷车不加油打不着，踩点油门...,技师说：这种情况建议把进气门和排气门间隙调整一下应该就好了，测量一下缸压是否正常，燃油压力是...,这种情况把发动机缸内积碳清洗一下，测量一下燃油压力是否正常，缸压是否正常，把喷油嘴更换了进行...


In [18]:
cws_model = "ltp_data_v3.4.0/cws.model"
pos_model = "ltp_data_v3.4.0/pos.model"
par_model = "ltp_data_v3.4.0/parser.model"
ner_model = "ltp_data_v3.4.0/ner.model"


def get_word_list(sentence=None,sentences=None,model=cws_model):
    #得到分词
    segmentor = Segmentor()
    segmentor.load(model)
    if sentences is not None:
        for i, s in enumerate(sentences):
            sentences[i] = list(segmentor.segment(s))
        return sentences
    else:
        word_list = list(segmentor.segment(sentence))
    segmentor.release()
    return word_list

def get_postag_list(word_list,model=pos_model):
    #得到词性标注
    postag = Postagger()
    postag.load(model)
    postag_list = list(postag.postag(word_list))
    postag.release()
    return postag_list

def get_parser_list(word_list,postag_list,model=par_model):
    #得到依存关系
    parser = Parser()
    parser.load(model)
    arcs = parser.parse(word_list,postag_list)
    arc_list = [(arc.head - 1,arc.relation) for arc in arcs]
    parser.release()
    return arc_list

In [21]:
sentences = list(df.Question) + list(df.Dialogue) + list(df.Report)
sentences = [el for el in sentences if type(el) == str]

In [22]:
sentences = get_word_list(sentences=sentences)

In [23]:
with open('gensim_input.txt', 'w') as f:
    for s in sentences:
        f.write(' '.join(s) + '\n')

In [24]:
from gensim.models.word2vec import LineSentence, Word2Vec

In [25]:
model = Word2Vec(corpus_file='gensim_input.txt', size=100, min_count=15, workers=8, sg=0, negative=5, iter=5)

In [26]:
model.save('word2vec.model')

In [27]:
words_count = list(model.wv.vocab.keys())

In [34]:
embedding_matrix = np.ones((len(words_count), 100))
for i, word in enumerate(words_count):
    embedding_matrix[i] = model.wv[word]

In [29]:
model.wv['车']

array([-0.9024219 , -2.6281116 ,  2.2922504 ,  2.8953884 , -0.6543503 ,
       -0.6721224 , -0.32666504, -2.6311643 ,  2.2705142 ,  2.8142567 ,
       -2.3997777 , -2.1317964 ,  0.38525718, -0.63711333, -2.8572416 ,
       -1.5664016 ,  2.008225  ,  0.07794845,  0.38996246,  2.0980325 ,
        2.480735  , -1.6455464 , -0.61552787,  1.4095308 ,  0.60903984,
        0.59424216, -1.6780497 ,  0.8891109 , -1.5162727 ,  0.6550863 ,
        1.4741954 , -1.8798184 , -1.2412721 ,  2.4950616 ,  0.15313336,
       -0.04052093, -0.8783153 , -1.7291406 ,  1.7668356 ,  1.1815282 ,
        4.754558  ,  1.125719  , -1.686371  , -3.2159874 , -5.014011  ,
       -1.0940299 , -0.65250045,  0.16762744,  1.8774986 ,  0.8787    ,
       -0.27753928,  4.111603  ,  0.48930058,  1.2807549 ,  0.04984166,
        2.6985362 , -2.2571907 , -0.5803061 ,  0.75837433, -0.32373333,
        3.32109   ,  0.1559487 , -2.0562637 ,  0.4975554 ,  1.6318727 ,
       -0.6096249 , -2.866278  ,  0.8384509 ,  2.1715012 ,  0.90

In [30]:
words_count

['方向',
 '，',
 '助力泵',
 '换',
 '了',
 '还是',
 '一样',
 '奔驰',
 '排气',
 '凸轮轴',
 '调节',
 '错误',
 '2010',
 '款',
 '宝马',
 'X1',
 '2011年',
 '出厂',
 '2.0',
 '排量',
 '通用',
 '变速箱',
 '原地',
 '挡',
 '位',
 '车辆',
 '闯动',
 '行驶',
 '升降档',
 '正常',
 '4',
 '轮',
 '离',
 '地',
 '无',
 '冲击感',
 '更换',
 '油',
 '12L',
 '改变',
 '。',
 '一',
 '辆',
 '2014年',
 '进口',
 '挡位',
 '也',
 '有',
 '这',
 '是',
 '什么',
 '情况',
 '哪里',
 '的',
 '问题',
 '3.0V6',
 '发动机',
 '号',
 '在',
 '位置',
 '照片',
 '最',
 '好',
 '！',
 '2012款',
 'c180',
 '怎么样',
 '维修',
 '保养',
 '动力',
 '值得',
 '吗',
 '科鲁兹',
 '旁边',
 '漏',
 '机油',
 '我',
 '要',
 '怎么',
 '才',
 '能',
 '知道',
 '车子',
 '断开',
 '电瓶',
 '电源',
 '之后',
 '不',
 '做',
 '节气门',
 '或',
 '防盗',
 '音响',
 '等',
 '重置',
 '还有',
 '重',
 '置',
 '所有',
 '车型',
 '都',
 '打开',
 '启动',
 '油门',
 '踩',
 '到底',
 '五',
 '秒',
 '重复',
 '几',
 '次',
 '就',
 '可以',
 '其他',
 '设备',
 '密码',
 '行',
 '昌河',
 '拆',
 '装',
 '长安',
 '35',
 '朝阳',
 '轮胎',
 '里面',
 '钢圈',
 '用',
 '自己',
 '多少',
 '钱',
 '外面',
 '吉利',
 '远景',
 '外',
 '球笼',
 '上',
 '那个',
 '大',
 '螺丝',
 '顺时针',
 '拧',
 '下来',
 '反',
 '五菱',
 '之',
 '光',
 '铁将军'

In [35]:
embedding_matrix

array([[-2.23305845, -0.20156799,  3.07305551, ..., -0.80874085,
         2.26590419, -1.81906295],
       [ 0.90768611,  0.3062312 ,  1.52877843, ..., -1.45252311,
         0.3009795 , -0.68515098],
       [ 0.17539899,  2.67784953,  1.15749228, ...,  0.56312746,
         1.56073689, -3.73715854],
       ...,
       [ 0.00899088,  0.055198  ,  0.44311333, ...,  0.92509729,
         0.07487664, -0.48667186],
       [-0.10242257, -0.23417504, -0.64697647, ..., -0.25296769,
        -0.10269587,  0.02698041],
       [ 1.26006901, -0.69341838, -0.86534935, ..., -0.19833702,
         0.51800954, -0.32576162]])

In [36]:
embedding_matrix.shape

(12655, 100)